In [9]:
import json
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from datetime import datetime, timezone

def load_data(clinics_file, treatment_rules_file):
    with open(clinics_file, 'r', encoding='utf-8') as f:
        clinics_data = json.load(f)
    with open(treatment_rules_file, 'r', encoding='utf-8') as f:
        treatment_rules = json.load(f)
    return clinics_data, treatment_rules

def preprocess_clinics(clinics_data):
    processed = []
    for region in clinics_data:
        for hospital in region["hospitals"]:
            for clinic in hospital["clinics"]:
                merged = {
                    **clinic,
                    "hospital_name": hospital["name"],
                    "hospital_type": hospital["type"],
                    "hospital_address": hospital["address"],
                    "region": region["region"],
                    "hospital_website": hospital.get("additional_info", {}).get("general_website", ""),
                    "emergency_contact": hospital.get("additional_info", {}).get("emergency_contact", "")
                }
                processed.append(merged)
    return processed

def is_pediatric_clinic(clinic):
    return "DOC" in clinic.get("categories", [])

def can_treat_in_roc(clinic, treatment_rules, diagnosis_info):
    """Check if ROC can treat this specific case"""
    if "ROC" not in clinic.get("categories", []):
        return False
    
    diagnosis_code = diagnosis_info.get("diagnosis_code")
    required_medication = diagnosis_info.get("required_medication")
    
    for rule in treatment_rules:
        for diagnosis in rule["diagnosis"]:
            specializations = diagnosis["diagnostical_group"]
            if isinstance(specializations, str):
                specializations = [specializations]
            
            if not any(spec in clinic.get("tumor_specializations", []) for spec in specializations):
                continue
            
            if diagnosis_code in diagnosis["disease_code"] or diagnosis_code in diagnosis["disease_code"].values():
                if not required_medication:
                    return True
                for med, brands in diagnosis["medication"].items():
                    if required_medication.lower() in [b.lower() for b in brands]:
                        return True
    return False

def filter_clinics(clinics, treatment_rules, criteria):
    filtered = []
    age = criteria.get("age", 18)
    
    for clinic in clinics:
        # Pediatric filter
        if age < 18:
            if not is_pediatric_clinic(clinic):
                continue
        else:
            if is_pediatric_clinic(clinic):
                continue
        
        # Other filters
        match = True
        
        if criteria.get("required_examination"):
            if not any(exam.lower() in [e.lower() for e in clinic.get("examinations", [])] 
               for exam in criteria["required_examination"]):
                match = False
        
        if criteria.get("hospitalization") is not None:
            if clinic.get("hospitalization") != criteria["hospitalization"]:
                match = False
        
        if criteria.get("insurances"):
            if not any(ins in clinic.get("insurances", []) for ins in criteria["insurances"]):
                match = False
        
        if criteria.get("region"):
            if clinic["region"] != criteria["region"]:
                match = False
        
        if match:
            # For non-pediatric cases with diagnosis info
            if age >= 18 and criteria.get("diagnosis_info"):
                # Check if ROC can treat this
                if can_treat_in_roc(clinic, treatment_rules, criteria["diagnosis_info"]):
                    filtered.append({**clinic, "priority": 1})  # Highest priority for ROC
                elif "KOC" in clinic.get("categories", []):
                    filtered.append({**clinic, "priority": 2})  # KOC as fallback
            else:
                filtered.append({**clinic, "priority": 0})  # Normal priority
    
    return filtered

def rank_clinics(clinics, user_location):
    geolocator = Nominatim(user_agent="clinic_finder_2025")
    user_coords = get_coordinates(geolocator, user_location)
    
    ranked = []
    current_date = datetime.now(timezone.utc).date()
    
    for clinic in clinics:
        clinic_coords = get_coordinates(geolocator, clinic["hospital_address"])
        distance = calculate_distance(user_coords, clinic_coords)
        
        try:
            appointment_date = datetime.strptime(clinic["nearest_appointment"], "%Y-%m-%d").date()
            days_until = (appointment_date - current_date).days
        except:
            days_until = float('inf')
        
        ranked.append({
            **clinic,
            "distance_km": round(distance, 2) if distance != float('inf') else "N/A",
            "days_until_appointment": days_until if days_until >= 0 else float('inf')
        })
    
    # Sort by: 1) Priority, 2) Distance, 3) Appointment date
    return sorted(ranked, key=lambda x: (
        x.get("priority", 0),
        x["distance_km"] if isinstance(x["distance_km"], (int, float)) else float('inf'),
        x["days_until_appointment"]
    ))

def get_coordinates(geolocator, address):
    if isinstance(address, dict):
        address_str = f"{address['street']}, {address['postal_code']} {address['city']}, {address['country']}"
    else:
        address_str = address
    location = geolocator.geocode(address_str)
    return (location.latitude, location.longitude) if location else None

def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).km if coord1 and coord2 else float('inf')

def main():
    clinics_data, treatment_rules = load_data(
        clinics_file="clinics.json",
        treatment_rules_file="roc_treatment_rules.json"
    )
    processed_clinics = preprocess_clinics(clinics_data)
    
    # Test case 1: Adult needing ROC treatment
    print("=== Adult needing ROC treatment (Breast cancer) ===")
    results = filter_clinics(
        processed_clinics,
        treatment_rules,
        {
            "age": 40,
            "diagnosis_info": {
                "diagnosis_code": "C50",
                "required_medication": "Herceptin"
            },
            "region": "Hlavní město Praha"
        }
    )
    ranked = rank_clinics(results, "Praha, Czech Republic")
    display_results(ranked)
    
    # Test case 2: Pediatric patient
    print("\n=== Pediatric patient (age 12) ===")
    results = filter_clinics(
        processed_clinics,
        treatment_rules,
        {
            "age": 12,
            "region": "Hlavní město Praha"
        }
    )
    ranked = rank_clinics(results, "Praha, Czech Republic")
    display_results(ranked)

    # Test case 3: Colonoscopy examination
    print("\n=== Colonoscopy examination ===")
    results = filter_clinics(
        processed_clinics,
        treatment_rules,
        {
            "age": 50,
            "required_examination": ["Colonoscopy"],
            "categories": ["CS"]
        }
    )
    ranked = rank_clinics(results, "Praha, Czech Republic")
    display_results(ranked)

def display_results(clinics):
    if not clinics:
        print("No matching clinics found")
        return
    
    for i, clinic in enumerate(clinics[:5], 1):
        print(f"\n#{i}: {clinic['name']} ({clinic['hospital_name']})")
        if clinic.get('priority') == 1:
            print("  • PRIORITY: ROC for this diagnosis")
        print(f"  • Type: {', '.join(clinic['categories'])}")
        if 'tumor_specializations' in clinic:
            print(f"  • Specializes in: {', '.join(clinic['tumor_specializations'])}")
        print(f"  • Distance: {clinic['distance_km']} km")
        print(f"  • Next appointment: {clinic['nearest_appointment']}")
        print(f"  • Address: {clinic['hospital_address']['street']}, {clinic['hospital_address']['city']}")
        print(f"  • Contact: {clinic['phone']}")


main()

=== Adult needing ROC treatment (Breast cancer) ===

#1: Regionální onkologické centrum (ROC) (Ústřední vojenská nemocnice – Vojenská fakultní nemocnice Praha)
  • PRIORITY: ROC for this diagnosis
  • Type: ROC
  • Specializes in: NKO, NHC, NPL, GIS, NPR, NLE, NOV
  • Distance: 4.15 km
  • Next appointment: 2025-11-25
  • Address: U Vojenské nemocnice 1200, Praha
  • Contact: +420 973 201 111

#2: Komplexní onkologické centrum (KOC) (Fakultní nemocnice Královské Vinohrady)
  • Type: KOC
  • Specializes in: 
  • Distance: 4.07 km
  • Next appointment: 2025-11-18
  • Address: Šrobárova 50, Praha
  • Contact: +420 267 161 111

#3: Komplexní onkologické centrum (KOC) (Fakultní nemocnice Motol v Praze)
  • Type: KOC, PSKOC
  • Specializes in: NPR, NKO, NPL, NOV
  • Distance: 6.01 km
  • Next appointment: 2025-11-10
  • Address: V Úvalu 84, Praha
  • Contact: +420 224 433 222

=== Pediatric patient (age 12) ===

#1: Dětské onkologické centrum (DOC) (Fakultní nemocnice Motol v Praze)
  • Type